<a href="https://colab.research.google.com/github/chang-min-dbs/remem/blob/python_summer/Day06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pygame
import random

pygame.init()
SCREEN_WIDTH = 800        #상수는 모두 대문자로 정의      #전역변수/상수 = 프로그램 전체적으로 참조하는 상,변수
SCREEN_HEIGHT = 600        #팝업창의 크기, 대문자로적은 이유는 상수역할을 하기위해(바뀌기 어려운 숫자)

#color 정의
WHITE = (255,255,255)  #??는 튜플이다.  #rgb는 0~255까지 있고 255가 될수록 진해짐
BLACK = (0,0,0)
GREEN = (0,255,0)
RED = (255,0,0)

BLOCK_SIZE = 20                                                                 #사용자 지정 1픽셀 값  #이것도 전역변수
INITIAL_SPEED = 15                                                              #스네이크의 초기 속도
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))                 #set_mode()에 ()와이트, 하이트들어가는 것 #파이게임 , 디스플레이는 모듈이다. 모듈은 클래스나 집합체이다.
pygame.display.set_caption("snake game")
clock = pygame.time.Clock()  #무슨요소??

#게임오버 만들기
class GameOverException(Exception):
    pass


#오브젝트 만들기 ---- 상속을 이용해 상위:게임클래스--하위 먹이,주인공
class GameObject:
    def __init__(self,x,y):
        self.x = x
        self.y = y

    def draw(self,color):
        pygame.draw.rect(screen,color, [self.x, self.y, BLOCK_SIZE, BLOCK_SIZE])  # 함수는 모듈??? ,리스트는 xy좌표와 너비,높이 이다.(블럭의)

class Food(GameObject):
    def __init__(self):                                                           #다시듣기
        x = random.randint(0, (SCREEN_WIDTH - BLOCK_SIZE) // BLOCK_SIZE) * BLOCK_SIZE
        y = random.randint(0, (SCREEN_HEIGHT - BLOCK_SIZE) // BLOCK_SIZE) *BLOCK_SIZE
        super().__init__(x,y)                                                      #곱하고 나누는이유는 랜덤하게 적은 개수만 뽑기 위해서


class Snake(GameObject):
    def __init__(self):
        super().__init__(SCREEN_HEIGHT // 2, SCREEN_HEIGHT // 2)                   #부모클래스를 통해 x,y값을 초기화 함// 정가운데에서 시작한다는 말
        self.direction = "RIGHT"
        self.body = []                                                              #픽셀이 스네이크 처럼 늘어날 때마다 좌표들을 가지고 있는 리스트 변수
        self.length = 1

    def grow(self):
        self.length += 1

    def draw(self,color):
        for block in self.body:
            pygame.draw.rect(screen, color, [block[0],block[1],BLOCK_SIZE, BLOCK_SIZE])      #여기 설명


    #블럭의 움직임 //방향을 결정할 인스탠스
    def move(self):
        if self.direction == 'UP':                                                # 오른쪽 대각선 밑이 최대(800,600) 이므로  오른쪽으로 이동 --> x 값 +, 밑으로 이동 --> y값 +
            self.y -= BLOCK_SIZE
        if self.direction == 'DOWN':
            self.y+= BLOCK_SIZE
        if self.direction == 'LEFT':
            self.x -= BLOCK_SIZE
        if self.direction == 'RIGHT':
            self.x += BLOCK_SIZE

        self.body.insert(0, [self.x, self.y])                                            #스네이크가 방향전환을 할 때20,0 -->20,20으로 바뀔텐데
        if len(self.body) > self.length:                                                  #자연스럽게 이동하는 효과를 보기 위해
            self.body.pop()                                                               #팝을하면 마지막 요소가 사라짐


    def change_dir(self,direction):
        if direction =='UP' and not self.direction =='DOWN':
            self.direction = 'UP'
        if direction =='DOWN' and not self.direction =='UP':                       #셀프 디렉션은 현재 위치. 현재 방향이 다운이 아니고 업일 때만 업으로
            self.direction = 'DOWN'
        if direction =='LEFT' and not self.direction =='RIGHT':                    #왼쪽에서 오른쪽으로 바로 전환 안되게끔 만든 것
            self.direction = 'LEFT'
        if direction =='RIGHT' and not self.direction =='LEFT':
            self.direction = 'RIGHT'


def game_loop():
    global INITIAL_SPEED                            #전역변수를 다른 범위 내에서 수정할 수 있게 해주는 선언
    game_over = False
    snake = Snake()                                 #instance 설정   / 캐틱터가 한개니깐>>?
    food = Food()

    while not game_over:                            #무한루프를 이용해 업데이트로 화면표시를 한다. -->근데 아직은 검은색으로만 나오게 설정한 것 # 어떨때 게임오버?
        for event in pygame.event.get():            # 시시각각 변하는 오브젝트를 눈에 보여주기 위함 + 화면이 켜저있을동안 이벤트를 감지하기 위해 반복문 사용+.get()가 이벤트를 캐치
            if event.type == pygame.QUIT:           #.get에서 여러 이벤트를 검사,(리스트같은 개념?으로 생각) 그중 이벤트 타임을 검사 -> 닫기 버튼 눌러 프로그램 종료한다.
                pygame.quit()                       #if문으로 이벤트 타입에 따라 병렬로 검사할 것이다.
            elif event.type == pygame.KEYDOWN:      #누르는 순간을 키 다온, 떼는 순간 키 업
                if event.key == pygame.K_UP:        #if문은 조건문이므로 == 이거 사용해야함
                    snake.change_dir ("UP")          #이건 할당연산자이므로 = 한개 사용해야함
                elif event.key ==  pygame.K_DOWN:
                    snake.change_dir("DOWN")
                elif event.key == pygame.K_LEFT:
                    snake.change_dir("LEFT")
                elif event.key == pygame.K_RIGHT:
                    snake.change_dir("RIGHT")
        if snake.x > SCREEN_WIDTH or snake.x < 0 or snake.y > SCREEN_WIDTH or snake.y < 0:
            raise GameOverException("Snake hit the boundry")

        if snake.x == food.x and snake.y == food.y:
            food = Food()                           #food를 다른공간에다 머시기 거시기~~~~
            INITIAL_SPEED += 2                      #먹었을 때 속도 올리기 위해/ 이렇게 메인프로세서에서 쓰기 위해선 특별한 선언이 필요-->전역변수이기 때문
            snake.grow()


        snake. move()                               #인스탠스 메서드가 실행되는 부분

        screen.fill(BLACK)
        snake.draw(RED)
        food.draw(WHITE)
        #go = GameObject(10,20)
        #go.draw(RED)
        pygame.display.update()

        clock.tick(INITIAL_SPEED)                   #무한루프 속도를 늦춰주는 역할, 초당 프레임 수 줄이는 역할임

if __name__ == "__main__":
    print("starting game...")                       #main process를 보기 좋게 정리..
    game_loop()                                     # a = 0    #지역변수 : 모듈내에서 참조하는 변수